# League Rosters



In [1]:
import pandas as pd
import os
from functions.my_leagues import *
import numpy as np

In [2]:
league = '792216196090019840'

In [3]:
players = pd.read_csv('Files/players.csv', dtype='object')

In [4]:
rosters = get_league_rosters(league_id=league)#key='ons')

In [5]:
teams = pd.DataFrame()
for index,row in rosters.iterrows():
    owner = row['owner_id']
    owner_list = [owner for p in row['players']]
    df = pd.DataFrame(owner_list, row['players']).reset_index()
    df.columns = ['player_id','owner_id']
    teams = teams.append(df)

print(teams.head())
             

  player_id            owner_id
0      1049  335516722092052480
1      1234  335516722092052480
2      1837  335516722092052480
3      2319  335516722092052480
4      2410  335516722092052480


In [6]:
league = get_league(league_id=league)#key='ons')

In [7]:
vor = pd.read_csv('Files/vor.csv', dtype='object')
vor = vor[vor['scoring_type'] == 'dynasty_2qb']
vor = vor[vor['league_size'] == '12']
vor['adp'] = vor['adp'].astype(float)
vor['vor'] = vor['vor'].astype(float)

In [8]:
league_data = league[['user_id','display_name']].merge(teams, left_on='user_id',right_on='owner_id',how='left').drop('owner_id',axis=1)
dataset = league_data.merge(players[['player_id','full_name','position','team']], on='player_id',how='left')
dataset = dataset.merge(vor[['player_id','adp','vor']], on='player_id',how='left')
#dataset = dataset.merge(draft_results[['player_id','round']], on='player_id',how='left')
dataset.to_csv('Files/dynasty_rosters.csv',index=False)

In [9]:
vor.describe()

,adp,vor
count,1398.000000,672.000000
mean,734.099785,-55.228661
std,353.706210,83.709510
min,1.600000,-296.260000
25%,460.425000,-108.425000
50%,999.000000,-37.000000
75%,999.000000,-4.000000
max,999.000000,179.900000


In [10]:
my_roster = dataset[dataset['display_name'] == 'windleypratt'].sort_values(by=['vor','adp'], ascending = [False,True])

In [11]:
my_roster

,user_id,display_name,player_id,full_name,position,team,adp,vor
7,335446335220121600,windleypratt,4034,Christian McCaffrey,RB,CAR,2.1,163.40
16,335446335220121600,windleypratt,6794,Justin Jefferson,WR,MIN,6.8,153.32
1,335446335220121600,windleypratt,2449,Stefon Diggs,WR,BUF,12.5,114.80
8,335446335220121600,windleypratt,4137,James Conner,RB,ARI,31.4,95.50
14,335446335220121600,windleypratt,5927,Terry McLaurin,WR,WAS,38.3,68.20
17,335446335220121600,windleypratt,6904,Jalen Hurts,QB,PHI,65.8,49.64
10,335446335220121600,windleypratt,5001,Dalton Schultz,TE,DAL,55.9,30.30
13,335446335220121600,windleypratt,5916,Darrell Henderson,RB,LAR,135.4,20.30
22,335446335220121600,windleypratt,8135,Treylon Burks,WR,TEN,100.5,3.60
11,335446335220121600,windleypratt,5086,Marquez Valdes-Scantling,WR,KC,138.7,0.00


In [12]:
available = players[['player_id','full_name','position','team', 'years_exp']].merge(vor[['player_id','adp','vor']], on='player_id',how='left').dropna(subset=['adp'])
available = available.merge(league_data, on='player_id', how='left')
available = available[~available['position'].isin(['DEF','K'])]
available = available[(available['display_name'] == 'windleypratt') | (available['display_name'].isnull())]

available.sort_values(by=['vor','adp'], ascending=[False,True]).to_csv('Files/available_players.csv', index=False)


In [13]:
dataset = dataset.sort_values(by=['user_id', 'vor'], ascending=[True,False])
ratings = dataset.groupby(['display_name', 'position']).head(3).sort_values(by=['position','user_id', 'vor'], ascending=[True, True, False]).to_csv('Files/team_ratings_data.csv', index=False)

In [30]:
limits = {'QB': 3, 'WR': 4, 'RB': 3, 'TE': 2}

In [31]:
ratings = pd.DataFrame()
for x in limits:
    n = limits.get(x)
    dfR = dataset[dataset['position'] == x].groupby(['display_name', 'position']).head(n)
    ratings = ratings.append(dfR)

In [32]:
ratings

,user_id,display_name,player_id,full_name,position,team,adp,vor
17,335446335220121600,windleypratt,6904,Jalen Hurts,QB,PHI,65.8,49.64
4,335446335220121600,windleypratt,3161,Carson Wentz,QB,WAS,251.5,-58.70
5,335446335220121600,windleypratt,3257,Jacoby Brissett,QB,CLE,999.0,-142.84
27,335516722092052480,itsmo,1234,Russell Wilson,QB,DEN,71.3,20.42
26,335516722092052480,itsmo,1049,Ryan Tannehill,QB,TEN,215.6,-46.36
...,...,...,...,...,...,...,...,...
234,792484068049686528,yejinkim4,6843,Albert Okwuegbunam,TE,DEN,127.4,0.00
251,792484337131032576,abnw24,1466,Travis Kelce,TE,KC,13.6,107.10
250,792484337131032576,abnw24,1339,Zach Ertz,TE,ARI,83.6,21.50
286,792485519014625280,Jprice08,5857,Noah Fant,TE,SEA,178.1,-31.00


In [33]:
ratings.groupby(['display_name','position'])['vor'].sum().reset_index().sort_values(by=['position', 'vor'], ascending=[True, False])

,display_name,position,vor
4,Mbalexan,QB,33.80
24,jspardue,QB,15.40
32,ndkauffm,QB,6.60
44,yejinkim4,QB,3.00
0,Jprice08,QB,-34.04
28,lakedawson,QB,-37.86
40,windleypratt,QB,-151.90
8,Pangle,QB,-183.30
12,abnw24,QB,-185.38
20,itsmo,QB,-195.00


,user_id,display_name,player_id
0,335446335220121600,windleypratt,2431
1,335446335220121600,windleypratt,2449
2,335446335220121600,windleypratt,2711
3,335446335220121600,windleypratt,2749
4,335446335220121600,windleypratt,3161
...,...,...,...
297,792485519014625280,Jprice08,8130
298,792485519014625280,Jprice08,8138
299,792485519014625280,Jprice08,8140
300,792485519014625280,Jprice08,8174
